In [1]:
using CSV, DataFrames, JuMP, Gurobi, StatsPlots, Random, Statistics

## Load in data

In [2]:
# Define sizes
#num_suppliers = 200 # i
#num_consumer_regions = 7 # j = c(i) 
#num_time = 10; # t 
#num_producer_regions = 5 # p(i)

# Parameters
# alpha
# delta

In [3]:
availability = CSV.read("availability.csv", DataFrame)
col_order = [1, 9, 8, 7, 6, 5, 4, 3, 2]
availability = availability[:, col_order]
availability = sort(availability, :Region)

Row,Region,yr_2016_shoes,yr_2017_shoes,yr_2018_shoes,yr_2019_shoes,yr_2020_shoes,yr_2021_shoes,yr_2022_shoes,yr_2023_shoes
,String15,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,AMERICAS,4898050,5500841,5313444,5454975,6324931,6109370,7043821,4628549
2,EMEA,727388,816895,789212,810092,939168,907185,1045899,687513
3,N ASIA,28864415,32415483,31312111,32145270,37273118,36002052,41507653,27275215
4,S ASIA,76623904,86051270,83122231,85334026,98946691,95572216,110187650,72405283
5,SE ASIA,100819493,113223358,109369617,112279721,130190750,125750659,144981558,95268541


In [4]:
demand = CSV.read("demand.csv", DataFrame)
replace!(demand.Region, "Europe, Middle East, and Africa" => "EMEA")
demand

Row,Region,2016,2017,2018,2019,2020,2021,2022,2023
,String31,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Asia Pacific,1.84351e7,2.06687e7,2.24933e7,2.27891e7,2.17006e7,2.30219e7,2.58658e7,2.85839e7
2,EMEA,4.32876e7,4.45665e7,5.04292e7,5.40172e7,5.05751e7,5.98283e7,6.34163e7,7.09013e7
3,Greater China,2.2309e7,2.50644e7,3.00086e7,3.65837e7,3.97854e7,4.93391e7,4.64893e7,4.66524e7
4,Latin America,6.71511e6,7.52871e6,8.19335e6,8.30106e6,7.90458e6,8.38586e6,9.42178e6,1.04119e7
5,North America,7.98197e7,8.31245e7,8.00172e7,8.62232e7,8.00773e7,9.99485e7,1.04961e8,1.27871e8


In [5]:
revenue = CSV.read("rev.csv", DataFrame)
replace!(revenue.Region, "Europe, Middle East, and Africa" => "EMEA")
rev_year = []
for i in 2:size(revenue)[2]
    col_sum = sum(revenue[:, i]) * 1e6
    append!(rev_year, col_sum)
end
rev_year = Array(rev_year);

In [6]:
holding_costs = CSV.read("inventory_holding_cost.csv", DataFrame)[:, 2:end]

Row,Year,holding_cost_per_shoe
,Int64,Float64
1,2016,43.92
2,2017,36.75
3,2018,30.22
4,2019,37.57
5,2020,43.73
6,2021,35.65
7,2022,36.63
8,2023,38.44


In [7]:
shipping_costs = CSV.read("transport_costs.csv", DataFrame);

In [8]:
shipping_mapping = shipping_costs[:, 1:2]
shipping_mapping = sort(shipping_mapping, [:producer_region, :consumer_region], rev=[false, false])
shipping_mapping = hcat(DataFrame(Row_Count=1:nrow(shipping_mapping)), shipping_mapping)
shipping_mapping = combine(groupby(shipping_mapping, [:producer_region, :consumer_region])) do sub_df
    DataFrame(Value_mean = first(sub_df.Row_Count))
end
shipping_mapping = unstack(shipping_mapping, :consumer_region, :Value_mean)
shipping_mapping = sort(shipping_mapping, :producer_region)

Row,producer_region,Asia Pacific,Greater China,EMEA,North America,Latin America
,String15,Int64?,Int64?,Int64?,Int64?,Int64?
1,AMERICAS,1,3,2,5,4
2,EMEA,6,8,7,10,9
3,N ASIA,11,13,12,15,14
4,S ASIA,16,18,17,20,19
5,SE ASIA,21,23,22,25,24


In [9]:
production_costs = CSV.read("cost.csv", DataFrame)
replace!(production_costs.Region, "Europe, Middle East, and Africa" => "EMEA");

In [10]:
production_costs

Row,Region,yr_2023_shoes,yr_2022_shoes,yr_2021_shoes,yr_2020_shoes,yr_2019_shoes,yr_2018_shoes,yr_2017_shoes,yr_2016_shoes
,String15,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,AMERICAS,81.465,38.903,45.4594,38.9553,47.7326,42.1257,45.144,42.5316
2,EMEA,76.8032,40.0918,46.9997,40.8797,44.1989,37.9323,38.5832,43.1429
3,N ASIA,70.5533,37.6785,49.5428,38.7469,46.6495,47.2958,42.2007,41.6894
4,S ASIA,82.0352,39.3142,47.2482,35.5651,42.4247,44.6127,35.3256,39.6973
5,SE ASIA,75.738,41.2462,48.1081,36.4332,44.048,47.8443,44.7181,40.4575


In [11]:
# Define sizes
num_suppliers = size(availability)[1] #i
num_consumer_regions = size(demand)[1] # j = c(i) 
num_time = size(availability)[2] - 1;  
num_producer_regions = size(availability)[1] # p(i)

5

### Set-up

In [12]:
# Sets
suppliers = 1:num_suppliers
consumer_regions = 1:num_consumer_regions
producer_regions = 1:num_producer_regions
years = 1:num_time
years_incl_zero = 0:num_time

0:8

In [13]:
# Parameters and input data
A = Matrix(availability[:, 2:end]) #5x8
D = Matrix(demand[:, 2:end]) #5x8
R = Array(rev_year) #8x1
H = Array(holding_costs[:, 2]) #8x1
T = Matrix(shipping_costs[:, 6:end]) #25x8
W = Matrix(shipping_mapping[:, 2:end]) #encodes the prodcuer/consumer region for T 5x5
C = Matrix(production_costs[:, 2:end]); #5x8

### Creating the model

In [61]:
# Define parameters (choose 1 value for now)
alpha = 1
cost_of_shoes = 116.5

116.5

In [68]:
function run_model(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0);
    @variable(model, S[suppliers, consumer_regions, years] >= 0);
    @variable(model, E[suppliers, years_incl_zero] >= 0);
    @variable(model, M[consumer_regions, years] >= 0);


    # Objective function
    @objective(model, Max, 
    sum(sum(sum((cost_of_shoes-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * cost_of_shoes * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}

    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model (generic function with 1 method)

In [70]:
model_simple, X_values_simple, S_values_simple, E_values_simple, M_values_simple = run_model(0.5, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Set parameter NonConvex to value 2
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 170 rows, 330 columns and 810 nonzeros
Model fingerprint: 0x99bcd7d3
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+05, 1e+08]
Presolve removed 50 rows and 10 columns
Presolve time: 0.00s
Presolved: 120 rows, 320 columns, 755 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.1819580e+35   3.840000e+32   1.181958e+05      0s
     141    1.1050097e+11   0.000000e+00   0.000000e+00      0s

Solved in 141 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.105009716e+11

User-callback calls 185, time 

In [74]:
net_profit_simple = objective_value(model_simple)
holding_cost_simple = sum(H[t]*sum(E_values_simple[i,t] for i in suppliers) for t in years)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_simple)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_simple)

Net profit: 1.1050097157426952e11
Holding cost: 8.66399627543e9


In [18]:
# Plot graphs

# Total costs over t
# Region

# Holding quantity over t

# Herfindalhs over t


In [19]:
# Plot graphs across alphas
# Ave Holding quantity over t
# Ave Herfindalhs over t


## Impose Integrality

In [71]:
function run_model_integer(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    @variable(model, M[consumer_regions, years] >= 0,  Int);


    # Objective function
    @objective(model, Max, 
    sum(sum(sum((cost_of_shoes-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * cost_of_shoes * M[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}


    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_integer (generic function with 1 method)

In [82]:
model_integer, X_values_integer, S_values_integer, E_values_integer, M_values_integer = run_model_integer(0.5, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 170 rows, 330 columns and 810 nonzeros
Model fingerprint: 0x15efd7ef
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+05, 1e+08]
Found heuristic solution: objective -1.00523e+11
Presolve removed 90 rows and 60 columns
Presolve time: 0.00s
Presolved: 80 rows, 270 columns, 505 nonzeros
Variable types: 0 continuous, 270 integer (0 binary)
Found heuristic solution: objective -8.40810e+10

Root relaxation: objective 1.105010e+11, 110 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Dep

In [85]:
net_profit_integer = objective_value(model_integer)
holding_cost_integer = sum(H[t]*sum(E_values_integer[i,t] for i in suppliers) for t in years)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_integer)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_integer)

Net profit: 1.1050097157426952e11
Holding cost: 8.66399627543e9


In [23]:
C

5×8 Matrix{Float64}:
 81.465   38.903   45.4594  38.9553  47.7326  42.1257  45.144   42.5316
 76.8032  40.0918  46.9997  40.8797  44.1989  37.9323  38.5832  43.1429
 70.5533  37.6785  49.5428  38.7469  46.6495  47.2958  42.2007  41.6894
 82.0352  39.3142  47.2482  35.5651  42.4247  44.6127  35.3256  39.6973
 75.738   41.2462  48.1081  36.4332  44.048   47.8443  44.7181  40.4575

#### Uncertainty in Demand
Define uncertainty based on underlying shoe prices

In [24]:
#Get Array of Sample Averages:
price_list = [115, 115, 115, 115, 120, 105, 125, 160, 90, 105]
avg_price = mean(price_list)
price_list_abs_dif = abs.(avg_price .- price_list)
sum(price_list_abs_dif)

111.0

In [25]:
2*(D[2,1]/avg_price * sum(price_list_abs_dif)) - D[2,1]

3.92003171416309e7

In [94]:
function run_model_integer_uncertainty(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    @variable(model, M[consumer_regions, years] >= 0,  Int);
    # Define uncertain parameters



    # Objective function
    @objective(model, Max, 
    sum(sum(sum((cost_of_shoes-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t]) * X[i,t] + H[t] * E[i,t] + alpha * cost_of_shoes * M[i,t] for i in suppliers) for t in years))

    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,0] == 0);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,0] == 0);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, time_in_inventory[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) >= E[i,t-1]); #goods can't be in inventory more than 1 year
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= 2*(D[j,t]/avg_price * sum(price_list_abs_dif)) - D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    @constraint(model, unmet_demand[j in consumer_regions, t in years], M[j,t] == D[j,t] - sum(S[i,j,t] for i in suppliers)); # M[i,t] = max{X[i,t] − X[i,t−1], 0}

    
    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_integer_uncertainty (generic function with 1 method)

In [95]:
model_uncertainty, X_values_uncertainty, S_values_uncertainty, E_values_uncertainty, M_values_uncertainty = run_model_integer_uncertainty(0.5, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 210 rows, 330 columns and 1050 nonzeros
Model fingerprint: 0x58050304
Variable types: 0 continuous, 330 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+05, 1e+08]
Found heuristic solution: objective -1.00523e+11
Presolve removed 95 rows and 60 columns
Presolve time: 0.00s
Presolved: 115 rows, 270 columns, 715 nonzeros
Variable types: 0 continuous, 270 integer (0 binary)
Found heuristic solution: objective -8.56672e+10

Root relaxation: objective 9.602212e+10, 110 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  D

In [96]:
net_profit_uncertainty = objective_value(model_uncertainty)
holding_cost_uncertainty = sum(H[t]*sum(E_values_uncertainty[i,t] for i in suppliers) for t in years)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_uncertainty)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_uncertainty)

Net profit: 9.602212494747112e10
Holding cost: 3.86479251152e9


## Compare Models

In [97]:
#Outputs
models = ["Simple", "Integer Solution", "Integer + Uncertainty in Demand"]
net_profit = [net_profit_simple, net_profit_integer, net_profit_uncertainty]
holding_cost = [holding_cost_simple, holding_cost_integer, holding_cost_uncertainty]
output = DataFrame(Model = models, NetProfit = net_profit, HoldingCost = holding_cost)

Row,Model,NetProfit,HoldingCost
,String,Float64,Float64
1,Simple,1.10501e11,8.664e9
2,Integer Solution,1.10501e11,8.664e9
3,Integer + Uncertainty in Demand,9.60221e10,3.86479e9


In [98]:
X_values_simple[:,2:end]

5×8 Matrix{Float64}:
      4.89805e6       5.50084e6  …       6.30152e6       4.25517e6
 727388.0        816895.0           961527.0        635770.0
      2.88644e7       3.24155e7          3.83837e7       2.33168e7
      3.52572e7       8.60513e7          9.41964e7       6.66096e7
      1.00819e8       5.61683e7          1.33376e8       9.1662e7

In [89]:
X_values_integer[:,2:end]

5×8 Matrix{Float64}:
      4.89805e6       5.50084e6  …       6.30152e6       4.25517e6
 727388.0        816895.0           961527.0        635770.0
      2.88644e7       3.24155e7          3.83837e7       2.33168e7
      3.52572e7       8.60513e7          9.41964e7       6.66096e7
      1.00819e8       5.61683e7          1.33376e8       9.1662e7

In [90]:
X_values_uncertainty[:,2:end]

5×8 Matrix{Float64}:
      4.89805e6       5.50084e6  …       6.30152e6       4.25517e6
 727388.0        816895.0           961527.0        635770.0
      2.88644e7       3.24155e7          3.83837e7       2.33168e7
      1.91522e7       8.60513e7          9.41964e7       6.66096e7
      1.00819e8       3.90826e7          1.33376e8       9.1662e7

## Add Supply Chain Shocks for 2021-2023
Reduce availability by a random number between 0-0.25

In [33]:
A_shock = A
for i in 1:5
    for j in 6:8
        Random.seed!(i+j)
        A_shock[i,j] = round(A_shock[i,j] * (1 - rand()/4))
    end
end
A_shock

5×8 Matrix{Int64}:
   4898050    5500841    5313444  …    5737011    6301519   4255168
    727388     816895     789212        811583     961527    635770
  28864415   32415483   31312111      33097796   38383735  23316833
  76623904   86051270   83122231      88379332   94196400  66609553
 100819493  113223358  109369617     107500789  133376412  91662033

Model 2016-2020 with original data

In [34]:
model_preshock, X_values_preshock, S_values_preshock, E_values_preshock, M_values_preshock = run_model_integer_uncertainty(1, 
    suppliers, consumer_regions, producer_regions, 1:5, 0:5, A[:, 1:5], D[:, 1:5], R[1:5], H[1:5], T[:, 1:5], W, C[:, 1:5]);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 210 columns and 510 nonzeros
Model fingerprint: 0x830e414d
Model has 25 quadratic objective terms
Variable types: 25 continuous, 185 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 1e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+05, 1e+08]
Found heuristic solution: objective -0.0000000
Presolve removed 40 rows and 45 columns
Presolve time: 0.00s
Presolved: 70 rows, 165 columns, 430 nonzeros
Variable types: 0 continuous, 165 integer (0 binary)
Found heuristic solution: objective 5.858368e+09

Root relaxation: objective 5.879566e+10, 68 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current 

In [35]:
net_profit_preshock = objective_value(model_preshock)
holding_cost_preshock = sum(H[t]*sum(E_values_preshock[i,t] for i in suppliers) for t in 1:5)
# sales_profit = net_profit + holding_cost

println("Net profit: ", net_profit_preshock)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_preshock)

Net profit: 5.879565518231283e10
Holding cost: 0.0


Model 2021-2023 with Supply Chain Shock

In [36]:
function run_model_shock(alpha, suppliers, consumer_regions, producer_regions, years, years_incl_zero, A, D, R, H, T, W, C)
    # Initialize model
    model = Model(Gurobi.Optimizer);

    # Decision variables
    # X_{i,t} = quantity of shoes produced by supplier i (in producer region p(i)) at time t
    # S_{i,j,t} = quantity of shoes sold to consumer region j at time t, that are produced by supplier i (in producer region p(i))
    # E_{i,t} = holding quantity of shoes by supplier i (in producer region p(i)) at time t
    # M_{i,t} = marginal cost of getting supplier i to produce shoes at time t
    @variable(model, X[suppliers, years_incl_zero] >= 0, Int);
    @variable(model, S[suppliers, consumer_regions, years] >= 0, Int);
    @variable(model, E[suppliers, years_incl_zero] >= 0,  Int);
    @variable(model, M[suppliers, years] == 0);
    # Define uncertain parameters



    # Objective function
    @objective(model, Max, 
    sum(sum(sum((116.5-T[W[i,j],t]) * S[i,j,t] for i in suppliers) for j in consumer_regions) for t in years) 
    - sum(sum((C[i,t] + alpha * M[i,t]) * X[i,t] + H[t] * E[i,t] for i in suppliers) for t in years))


    # Constraints
    @constraint(model, initial_production_constraint[i in suppliers], X[i,5] == X_values_preshock[i, 5]);
    @constraint(model, initial_excess_constraint[i in suppliers], E[i,5] == E_values_preshock[i, 5]);
    @constraint(model, production_sold_excess_relationship[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) == X[i,t] + E[i,t-1] - E[i,t]);
    @constraint(model, time_in_inventory[i in suppliers, t in years], sum(S[i,j,t] for j in consumer_regions) >= E[i,t-1]); #goods can't be in inventory more than 1 year
    @constraint(model, demand_constraint[j in consumer_regions, t in years], sum(S[i,j,t] for i in suppliers) <= 2*(D[j,t]/avg_price * sum(price_list_abs_dif)) - D[j,t]);
    @constraint(model, supply_production_constraint[i in suppliers, t in years], X[i,t] <= A[i,t]);
    #@constraint(model, marginal_cost_constraint[i in suppliers, t in years], M[i,t] >= X[i,t] - X[i,t-1]); # M[i,t] = max{X[i,t] − X[i,t−1], 0}

    
    #set_optimizer_attribute(model, "NonConvex", 2);
    optimize!(model);

    # Get values
    X_values = Matrix(value.(X))
    S_values = value.(S)
    E_values = Matrix(value.(E))
    M_values = Matrix(value.(M))

    return model, X_values, S_values, E_values, M_values
end

run_model_shock (generic function with 1 method)

In [37]:
model_shock, X_values_shock, S_values_shock, E_values_shock, M_values_shock = run_model_shock(1, 
    suppliers, consumer_regions, producer_regions, 6:8, 5:8, A_shock, D, R, H, T, W, C);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-22
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 70 rows, 130 columns and 310 nonzeros
Model fingerprint: 0x2cf5b6f6
Model has 15 quadratic objective terms
Variable types: 15 continuous, 115 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 1e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+05, 1e+08]
Found heuristic solution: objective -0.0000000
Presolve removed 30 rows and 35 columns
Presolve time: 0.00s
Presolved: 40 rows, 95 columns, 240 nonzeros
Variable types: 0 continuous, 95 integer (0 binary)
Found heuristic solution: objective 1.464491e+10

Root relaxation: objective 4.708917e+10, 35 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Nod

In [38]:
net_profit_shock = objective_value(model_shock)
holding_cost_shock = sum(H[t]*sum(E_values_shock[i,t] for i in suppliers) for t in 1:3)
# sales_profit = net_profit + jjholding_cost

println("Net profit: ", net_profit_shock)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_cost_shock)

Net profit: 4.708917368119233e10
Holding cost: 1.41081547194e9


In [39]:
#Combined Profit
net_profit_pre_post = net_profit_shock + net_profit_preshock
holding_pre_post = holding_cost_shock + holding_cost_preshock

println("Net profit: ", net_profit_pre_post)
# println("Sales profit: ", sales_profit)
println("Holding cost: ", holding_pre_post)

Net profit: 1.0588482886350516e11
Holding cost: 1.41081547194e9


## Code from HW1 (may be useful)

In [40]:
# Create a heatmap of (number of units of product i manufactured, week t)  and the 
heatmap(X_values, axis=true, color=:viridis, c=:auto, size=(800, 600), 
        xlabel="Week", ylabel="Number of units produced", title="Number of units of product i manufactured at week t")

LoadError: UndefVarError: `X_values` not defined

In [41]:
# Create a heatmap of (number of units of product i stored in inventory, week t)
heatmap(E_values, axis=true, color=:viridis, c=:auto, size=(800, 600), 
        xlabel="Week", ylabel="Number of units stored in inventory", title="Number of units of product i stored in inventory at week t")

LoadError: UndefVarError: `E_values` not defined

Interpretation: 
- It makes sense to produce excess units of a product if there is unsatisfied demand some time in the future (that cannot be met by production at any later week), and net profit still exceeds the total holding cost.
- From the heatmap of excess units (holdings), we see that few products and weeks have positive holdings. This implies that the above condition is not met most of the time.


## 3(d)

In [42]:
week30_unused_materials = Vector{Float64}(undef, size(materials,1)) 
print("Materials fully utilized in week 30: ")
for j in materials
    week30_unused_materials[j] = A[j,30] - sum((R[j,i] * X_values[i,30]) for i in products)
    if abs(week30_unused_materials[j]) < 1e-10
        week30_unused_materials[j] = 0
        print(j, " ")
    end
end
println("\nNumber of materials fully utilized in week 30 (out of 500): ", count(x -> x == 0, week30_unused_materials))
println("Number of materials not fully utilized in week 30 (out of 500): ", count(x -> x > 0, week30_unused_materials))

LoadError: UndefVarError: `materials` not defined

In [43]:
week30_sold = X_values[:,30] + E_values[:,29] - E_values[:,30]
week30_demand = D[:,30]
week30_unfulfilled_demand = week30_demand - week30_sold
print("Products with demand fully satisfied in week 30: ")
for i in products
    if abs(week30_unfulfilled_demand[i]) < 1e-10
        week30_unfulfilled_demand[i] = 0
        print(i, " ")
    end
end
println("\nNumber of products with demand fully satisfied in week 30 (out of 100): ", count(x -> x == 0, week30_unfulfilled_demand))
println("Number of products with demand not fully satisfied in week 30 (out of 100): ", count(x -> x > 0, week30_unfulfilled_demand))

LoadError: UndefVarError: `X_values` not defined